In [248]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO
import string
from datetime import datetime


In [249]:
doc = "output-4-pandas.csv"
if('-35-' in doc):
    qtr_date = 'December 31, 2020'
else:
    qtr_date = '22132'
pd_csv = pd.read_csv(doc)
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
# Find out where portfolio company is
for col in pd_csv.columns:
    pd_csv[col] = pd_csv[col].astype(str).str.replace('  ', ' ')
    portfolio_index_list = pd_csv.index[pd_csv[col].astype(str).str.contains("Company")].to_list()
    if (len(portfolio_index_list) > 0):
        break
portfolio_index = portfolio_index_list[0]
# if(len(portfolio_index_list)>1):
#     pd_csv.drop(portfolio_index_list[1], inplace=True)

# Checking if all the column are in the row with portfolio and adding them if they are not.
for j in range(pd_csv.shape[1]):
    if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
        pd_csv.iat[portfolio_index, j] = 'Principal'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
        pd_csv.iat[portfolio_index, j] = 'Cost'
    elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
        pd_csv.iat[portfolio_index, j] = 'Fair Value'
    elif ('Principal' in str(pd_csv.iat[portfolio_index - 1, j])):
        pd_csv.iat[portfolio_index, j] = 'Principal'

print("Portofolio", portfolio_index)
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Company').any() == True):
    pd_csv = pd_csv.iloc[portfolio_index:]


Portofolio 0


In [250]:
pd_csv

,0,1,2,3,4,5,6,7,8,9,10,11
0,Portfolio Company (1),NaN,Business Description,NaN,Type of Investment (2) (3),NaN,Principal (4),NaN,Cost (4),NaN,Fair Value,NaN
1,Control Investments (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Bond-Coat, Inc.",NaN,Casing and Tubing Coating Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nan,NaN,NaN,NaN,"12% Secured Debt (Maturity - December 28, 2017)",NaN,14750,NaN,14557,NaN,14557,NaN
4,nan,NaN,NaN,NaN,Common Stock (Fully diluted 43.4%),NaN,NaN,NaN,6350,NaN,6350,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
472,(9),Index based floating interest rate is subject ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,(10),Private Loans portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,(11),Middle Market portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,(12),Other Portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
#Identifying levels in the dataframe
listOfLevels = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (pd.notnull(pd_csv.iat[i, j])):
            # print(pd_csv.iat[i, j])
            if ((('investments' in str(pd_csv.iat[i, j])) or ('Investments' in str(pd_csv.iat[i, j]))) and
                    ('Subtotal' not in pd_csv.iat[i, j]) and ('Total' not in pd_csv.iat[i, j]) and
                    ('Inc' not in pd_csv.iat[i, j]) and ('Fund' not in pd_csv.iat[i, j]) and 
                    ('LLC' not in pd_csv.iat[i, j]) and ('L.P' not in pd_csv.iat[i, j])  and 
                    ('CMS' not in pd_csv.iat[i, j]) and ('Holdings' not in pd_csv.iat[i, j])  ):
                listOfLevels.append(i)

In [252]:
listOfLevels

[1, 138, 217, 465, 468, 469, 470]

In [253]:
listOfInvestments = list()

if (pd_csv.iloc[0].str.contains('Type of Investment').any() == False):
    for i in range(pd_csv.shape[0]):
        for j in range(pd_csv.shape[1]):
            if (pd.notnull(pd_csv.iat[i, j])):
                # print(pd_csv.iat[i, j])
                if ((('Senior Secured Notes' in str(pd_csv.iat[i, j])) or
                     ('Senior Secured Second Lien Term Loans' in str(pd_csv.iat[i, j])) or
                     ('Warrants/Equity' in str(pd_csv.iat[i, j])))
                        and ('Total' not in str(pd_csv.iat[i, j]))):
                    listOfInvestments.append(i)
    # Adding the Levels to an array
    InvestmentValues = []
    InvestmentValues.append("Type of Investments")
    InvestmentValues.append("")
    for i in range(len(listOfInvestments)):
        beginIndex = listOfInvestments[i]
        if (i == len(listOfInvestments) - 1):
            endIndex = len(pd_csv)
        else:
            endIndex = listOfInvestments[i + 1]
        value = pd_csv.iat[beginIndex, 0]
        value = ''.join([i for i in value if not i.isdigit()])
        value = re.sub("[^\P{P}-]+", "", value)
        for x in range(beginIndex, endIndex):
            InvestmentValues.append(value)
            # Adding the array to the df
    # pd_csv.insert(1, 'Type of Investments', InvestmentValues)

In [254]:
pd_csv

,0,1,2,3,4,5,6,7,8,9,10,11
0,Portfolio Company (1),NaN,Business Description,NaN,Type of Investment (2) (3),NaN,Principal (4),NaN,Cost (4),NaN,Fair Value,NaN
1,Control Investments (5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Bond-Coat, Inc.",NaN,Casing and Tubing Coating Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nan,NaN,NaN,NaN,"12% Secured Debt (Maturity - December 28, 2017)",NaN,14750,NaN,14557,NaN,14557,NaN
4,nan,NaN,NaN,NaN,Common Stock (Fully diluted 43.4%),NaN,NaN,NaN,6350,NaN,6350,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
472,(9),Index based floating interest rate is subject ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,(10),Private Loans portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,(11),Middle Market portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,(12),Other Portfolio investment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
len(pd_csv)

477

In [256]:
remove = string.punctuation
remove = remove.replace("-", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern

# Adding the Levels to an array
newValues = []
newValues.append("Levels")

for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if (i == len(listOfLevels) - 1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    value = ''.join([i for i in value if not i.isdigit()])
    value = re.sub("[^\P{P}-]+", "", value)
    print("Begin",beginIndex)
    print("End",endIndex)
    for x in range(beginIndex, endIndex):
        newValues.append(value)

#print(newValues)
if(len(newValues)<(len(pd_csv))):
    newValues.insert(0, 'Levels')
# Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)



Begin 1
End 138
Begin 138
End 217
Begin 217
End 465
Begin 465
End 468
Begin 468
End 469
Begin 469
End 470
Begin 470
End 477


In [257]:
# Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            # print(pd_csv.iloc[i,:])
            subtotal_list.append(i)

print("Subtotal", subtotal_list)
# Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list], inplace=True)

# Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if ('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
            
print("Total list",total_list)
pd_csv.drop(pd_csv.index[total_list], inplace=True)
# Dropping everything after the total rows
try:
    pd_csv = pd_csv.iloc[:total_list[-1]]
except:
    print("Total not found")
    
for i in range(5):
    for j in range(pd_csv.shape[1]):
        if ('TypeofInvestment' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            type_of_investment_index = j
            break
        elif ('Industry' in str(pd_csv.iat[i,j]).translate({ord(c): None for c in string.whitespace})):
            type_of_investment_index = j
print("type_of_investment_index",type_of_investment_index)

Subtotal [137, 216, 461]
Total list [461]
type_of_investment_index 5


In [258]:
pd_csv[-60:]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
403,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,11% Current / 2.50% PIK Secured Debt (Maturity...,NaN,4000,NaN,4027,NaN,4027,NaN
405,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,Warrants (Fully diluted 0.05%),NaN,NaN,NaN,53,NaN,53,NaN
406,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4080,NaN,4080,NaN
407,Non-ControlNon-Affiliate Investments,"Shale-Inland Holdings, LLC (11)",NaN,"Distributor of Pipe, Valves, and Fittings",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,"8.75% Bond (Maturity - November 15, 2019)",NaN,3000,NaN,3000,NaN,3150,NaN
409,Non-ControlNon-Affiliate Investments,"Sonneborn, LLC (11)",NaN,Specialty Chemicals Manufacturer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,"LIBOR Plus 5.00%, Current Coupon 6.50%, Secure...",NaN,2970,NaN,2919,NaN,3029,NaN
411,Non-ControlNon-Affiliate Investments,"Sotera Defense Solutions, Inc. (11)",NaN,Defense Industry Intelligence Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,Non-ControlNon-Affiliate Investments,nan,NaN,NaN,NaN,"LIBOR Plus 6.00%, Current Coupon 7.50%, Secure...",NaN,9171,NaN,8839,NaN,9080,NaN


In [259]:
pd_csv.isnull().sum()/len(pd_csv)

Levels    0.000000
0         0.000000
1         1.000000
2         0.668113
3         1.000000
4         0.464208
5         1.000000
6         0.648590
7         1.000000
8         0.336226
9         1.000000
10        0.336226
11        1.000000
dtype: float64

In [260]:
#print(type_of_investment_index)
# Adding company for the middle values where it might be missed out
for i in range(2, pd_csv.shape[0]):
    for j in range(type_of_investment_index):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k])):
                                # print("Alpha Check",pd_csv.iat[i,k])
                                row_alphabet_check = True
                                break
                        if (row_alphabet_check):
                            # print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]



In [261]:
pd_csv[-60:]

,Levels,0,1,2,3,4,5,6,7,8,9,10,11
403,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,11% Current / 2.50% PIK Secured Debt (Maturity...,NaN,4000,NaN,4027,NaN,4027,NaN
405,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,Warrants (Fully diluted 0.05%),NaN,NaN,NaN,53,NaN,53,NaN
406,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,NaN,NaN,NaN,NaN,4080,NaN,4080,NaN
407,Non-ControlNon-Affiliate Investments,"Shale-Inland Holdings, LLC (11)",NaN,"Distributor of Pipe, Valves, and Fittings",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,Non-ControlNon-Affiliate Investments,"Shale-Inland Holdings, LLC (11)",NaN,"Distributor of Pipe, Valves, and Fittings",NaN,"8.75% Bond (Maturity - November 15, 2019)",NaN,3000,NaN,3000,NaN,3150,NaN
409,Non-ControlNon-Affiliate Investments,"Sonneborn, LLC (11)",NaN,Specialty Chemicals Manufacturer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,Non-ControlNon-Affiliate Investments,"Sonneborn, LLC (11)",NaN,Specialty Chemicals Manufacturer,NaN,"LIBOR Plus 5.00%, Current Coupon 6.50%, Secure...",NaN,2970,NaN,2919,NaN,3029,NaN
411,Non-ControlNon-Affiliate Investments,"Sotera Defense Solutions, Inc. (11)",NaN,Defense Industry Intelligence Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,Non-ControlNon-Affiliate Investments,"Sotera Defense Solutions, Inc. (11)",NaN,Defense Industry Intelligence Services,NaN,"LIBOR Plus 6.00%, Current Coupon 7.50%, Secure...",NaN,9171,NaN,8839,NaN,9080,NaN


In [262]:
# Identifying index of maturity
i = 0
# Extracting the column data after shares/warrants/principal
try:
    pd_csv_principal = pd_csv.iloc[:, type_of_investment_index:]
    pd_csv_principal = pd_csv_principal.replace("%",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b",np.nan)
    pd_csv_principal = pd_csv_principal.replace("\u200b.1",np.nan)

    # Sweeping everyhing to left in extracted df
    pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))
except:
    print("no Maturity")

In [263]:
pd_csv_principal[-60:]

,Unnamed: 0,4,5,6,7,8,9,10,11
401,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,404,11% Current / 2.50% PIK Secured Debt (Maturity...,4000,4027,4027,NaN,NaN,NaN,NaN
403,405,Warrants (Fully diluted 0.05%),53,53,NaN,NaN,NaN,NaN,NaN
404,406,4080,4080,NaN,NaN,NaN,NaN,NaN,NaN
405,407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,408,"8.75% Bond (Maturity - November 15, 2019)",3000,3000,3150,NaN,NaN,NaN,NaN
407,409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,410,"LIBOR Plus 5.00%, Current Coupon 6.50%, Secure...",2970,2919,3029,NaN,NaN,NaN,NaN
409,411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,412,"LIBOR Plus 6.00%, Current Coupon 7.50%, Secure...",9171,8839,9080,NaN,NaN,NaN,NaN


In [264]:
pd_csv_principal = pd_csv_principal.iloc[:,1:]
# Dropping columns with Nulls
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[0] + 1)
pd_csv_principal = pd_csv_principal.dropna(axis=1,
                                           thresh=min_count)
# pd_csv_principal.update(
#     pd_csv_principal.loc[pd_csv_principal.iloc[:, -1].isna(), :].shift(axis=1).replace(np.nan, 'na')
# )
pd_csv_principal.replace('na', np.nan, inplace=True)
pd_csv = pd_csv.iloc[:, :type_of_investment_index].reset_index(drop=True)



In [265]:
# Concatting everything together
pd_csv = pd.concat([pd_csv, pd_csv_principal], axis=1)


In [266]:
pd_csv[-60:]

,Levels,0,1,2,3,4,5,6,7
401,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,NaN,NaN,NaN,NaN
402,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,11% Current / 2.50% PIK Secured Debt (Maturity...,4000,4027,4027
403,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,Warrants (Fully diluted 0.05%),53,53,NaN
404,Non-ControlNon-Affiliate Investments,"SAExploration, Inc. (11) (13)",NaN,Geophysical Services Provider,NaN,4080,4080,NaN,NaN
405,Non-ControlNon-Affiliate Investments,"Shale-Inland Holdings, LLC (11)",NaN,"Distributor of Pipe, Valves, and Fittings",NaN,NaN,NaN,NaN,NaN
406,Non-ControlNon-Affiliate Investments,"Shale-Inland Holdings, LLC (11)",NaN,"Distributor of Pipe, Valves, and Fittings",NaN,"8.75% Bond (Maturity - November 15, 2019)",3000,3000,3150
407,Non-ControlNon-Affiliate Investments,"Sonneborn, LLC (11)",NaN,Specialty Chemicals Manufacturer,NaN,NaN,NaN,NaN,NaN
408,Non-ControlNon-Affiliate Investments,"Sonneborn, LLC (11)",NaN,Specialty Chemicals Manufacturer,NaN,"LIBOR Plus 5.00%, Current Coupon 6.50%, Secure...",2970,2919,3029
409,Non-ControlNon-Affiliate Investments,"Sotera Defense Solutions, Inc. (11)",NaN,Defense Industry Intelligence Services,NaN,NaN,NaN,NaN,NaN
410,Non-ControlNon-Affiliate Investments,"Sotera Defense Solutions, Inc. (11)",NaN,Defense Industry Intelligence Services,NaN,"LIBOR Plus 6.00%, Current Coupon 7.50%, Secure...",9171,8839,9080


In [267]:

# Sweeping to left for member units
# pd_csv_sweep_left = pd_csv.iloc[:, :(principal_index + 2)]
# pd_csv_sweep_left = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_sweep_left.to_csv())))
# pd_csv_sweep_right = pd_csv.iloc[:, principal_index + 2:]
# pd_csv = pd.concat([pd_csv_sweep_left, pd_csv_sweep_right], axis=1)

perc = 100  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# Dropping columns which have 100% NaN
pd_csv = pd_csv.dropna(axis=1,
                       thresh=min_count)

new_header = pd_csv.iloc[0]  # grab the first row for the header
#new_header = new_header.str.replace("Shares/Warrants/Principal","Shares Warrants Principal")
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

C:\Users\jaiso\AppData\Local\Temp\ipykernel_16884\3648037571.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [268]:
pd_csv.isnull().sum()/len(pd_csv)

0
Levels                  0.000000
Portfolio Company       0.000000
Business Description    0.002174
Type of Investment      0.336957
Principal               0.336957
Cost                    0.465217
Fair Value              0.650000
dtype: float64

In [269]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
1,Control Investments,Control Investments (5),NaN,NaN,NaN,NaN,NaN
2,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,NaN,NaN,NaN,NaN
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),6350,6350,NaN
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
6,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,NaN,NaN,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%) (8),42,3740,NaN
9,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,342,4040,NaN,NaN
10,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,NaN,NaN,NaN,NaN


In [270]:
pd_csv = pd_csv.replace("~", ",", regex=True)
#pd_csv = pd_csv.iloc[:, 1:]

# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
maturity_cols = [col for col in pd_csv.columns if 'Maturity' in col]
if(len(maturity_cols)<1):
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 3)
else:
    pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 5)




In [271]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),6350,6350,NaN
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%) (8),42,3740,NaN
9,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,342,4040,NaN,NaN
11,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
12,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),1193,3380,NaN
13,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),1177,1560,NaN
14,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,10298,13033,NaN,NaN


In [272]:
for i in range(int(pd_csv.shape[0])):
    for j in range(pd_csv.shape[1]):
        try:
            pd_csv.iat[i,j] = re.sub(r"\(\d+\)", "", pd_csv.iat[i,j])
            if(pd_csv.iat[i,j] == ''):
                pd_csv.iat[i,j] = np.nan
        except:
            print("Bracket digits not removed")

Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
Bracket digits not removed
B

In [273]:
if (qtr_date == 'December 31, 2020'):
    fair_cols = [col for col in pd_csv.columns if 'Fair' in col]
    pd_csv = pd_csv.reset_index(drop=True)

    for i in range(int(pd_csv.shape[0])):
        for j in range(int(pd_csv.shape[1])-4,int(pd_csv.shape[1])):
            #print(i)
            #print(pd_csv.iat[i,j])
            if(pd.isnull(pd_csv.iat[i,j])):
                1
                #print("")
            elif('-' in pd_csv.iat[i,j]):
                if(len(pd_csv.iat[i,j])>1):
                    pd_csv.iat[i,j] = np.nan

In [274]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),6350,6350,NaN
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),42,3740,NaN
9,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,342,4040,NaN,NaN
11,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
12,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),1193,3380,NaN
13,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),1177,1560,NaN
14,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,10298,13033,NaN,NaN


In [275]:
# pd_csv = pd_csv.reset_index(drop=True)
# if (len(maturity_cols) > 0):
#     fair_cols = [col for col in pd_csv.columns if 'Fair' in col]
#     type_of_inv_cols = [col for col in pd_csv.columns if 'Type of' in col]
#     #Checking if Fair value has String
# #     try:
# #         pd_csv[fair_cols[0]] = pd.to_numeric(pd_csv[fair_cols[0]]) 
# #     except ValueError:
# #         mask = pd.to_numeric(pd_csv[fair_cols[0]].fillna('0'), errors='coerce').isna()
# #         values_in_fair = mask[mask].index.to_list()
#     values_in_fair = pd_csv[type_of_inv_cols[0]].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).index.to_list()
#     if(len(values_in_fair)>0 and  qtr_date == 'December 31, 2020'):
#         print("Inside")
#         i = 0
#         for i in values_in_fair:
#             #print(pd_csv.loc[i,fair_cols[0]])
#             if( (pd_csv.loc[i,fair_cols[0]] == '-') or (pd_csv.loc[i,fair_cols[0]] == ' ') or 
#                (pd.isna(pd_csv.loc[i,fair_cols[0]])) ):
#                 values_in_fair.remove(i)
#         try:
#             type_of_investment = pd_csv.loc[values_in_fair,fair_cols[0]]
#             # Shifting the rows to right from type of investment
#             pd_csv.loc[values_in_fair, type_of_inv_cols[0]:] = pd_csv.loc[values_in_fair, type_of_inv_cols[0]:].shift(1,
#                                                                                                                           axis=1)
#             #Putting type of investment back into type of investment
#             pd_csv.loc[values_in_fair,type_of_inv_cols[0]] = type_of_investment
#         except:
#             print("Shifting Failed")

In [276]:
value_per_as_nan_list = pd_csv.index[(pd_csv.iloc[:,-1].isnull()) & (pd_csv.iloc[:,-2].notnull())].to_list()
csv_columns = pd_csv.columns
csv_columns = csv_columns[-4:]
pd_csv.loc[value_per_as_nan_list,[csv_columns[2],csv_columns[3]]] = pd_csv.loc[value_per_as_nan_list,[csv_columns[3],csv_columns[2]]].values
pd_csv.loc[value_per_as_nan_list,[csv_columns[1],csv_columns[2]]] = pd_csv.loc[value_per_as_nan_list,[csv_columns[2],csv_columns[1]]].values
pd_csv.loc[value_per_as_nan_list,[csv_columns[0],csv_columns[1]]] = pd_csv.loc[value_per_as_nan_list,[csv_columns[1],csv_columns[0]]].values

In [277]:
#Shifting the value when maturity is NaN
try:
    maturity_nan_index = pd_csv.index[~pd_csv['Maturity'].astype(str).str.contains("/")].to_list()
except:
    print("Maturity not present")

Maturity not present


In [278]:
maturity_amount_cols = pd_csv.columns[-5:-3]
maturity_amount_cols

Index(['Business Description', 'Type of Investment  '], dtype='object', name=0)

In [279]:
#maturity_nan_index

In [280]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,NaN,Common Stock (Fully diluted 43.4%),6350,6350
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,NaN,Member Units (Fully diluted 41.0%),42,3740
9,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,342,4040,NaN,NaN
11,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
12,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,NaN,Warrants (Fully diluted 21.3%),1193,3380
13,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,NaN,Common Stock (Fully diluted 9.8%),1177,1560
14,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,10298,13033,NaN,NaN


In [281]:
try:
    for ind in maturity_nan_index:
        if( str(pd_csv.loc[ind,'Industry'])=='nan' and str(pd_csv.loc[ind,'Interest'])=='nan' ):
            print("Last")
            if(str(pd_csv.loc[ind,'Maturity']) == 'Industry'):
                print("IF")
                pd_csv.loc[ind:,['Maturity','Industry']] = pd_csv.loc[ind:,['Industry','Maturity']].values
                break
            else:
                print("Else")
                pd_csv.loc[ind, [maturity_amount_cols[0],maturity_amount_cols[1]]] = pd_csv.loc[ind, [maturity_amount_cols[1],maturity_amount_cols[0]]].values
        else:
            pd_csv.loc[ind, [maturity_amount_cols[0],maturity_amount_cols[1]]] = pd_csv.loc[ind, [maturity_amount_cols[1],maturity_amount_cols[0]]].values

except:
    print("Error when switching for rows after second header")

Error when switching for rows after second header


In [282]:
pd_csv

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,NaN,Common Stock (Fully diluted 43.4%),6350,6350
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,NaN,Member Units (Fully diluted 41.0%),42,3740
...,...,...,...,...,...,...,...
454,Non-ControlNon-Affiliate Investments,Wireco Worldgroup Inc.,Manufacturer of Synthetic Lifting Products,"LIBOR Plus 4.75%, Current Coupon 6.00%, Secure...",2488,2466,2522
456,Non-ControlNon-Affiliate Investments,Zilliant Incorporated,Price Optimization and Margin Management Solut...,"12% Secured Debt (Maturity - June 15, 2017)",8000,6911,6911
457,Non-ControlNon-Affiliate Investments,Zilliant Incorporated,Price Optimization and Margin Management Solut...,NaN,Warrants (Fully diluted 3.0%),1071,1071
458,Non-ControlNon-Affiliate Investments,Zilliant Incorporated,Price Optimization and Margin Management Solut...,7982,7982,NaN,NaN


In [283]:
principal_cols = [col for col in pd_csv.columns if 'Principal' in col]
type_of_inv_cols = [col for col in pd_csv.columns if 'Type of' in col]
maturity_cols = [col for col in pd_csv.columns if 'Maturity' in col]
type_of_inv_cols

['Type of Investment  ']

In [284]:
len(maturity_cols)

0

In [285]:
try:
    pd_csv[principal_cols[0]] = pd.to_numeric(pd_csv[principal_cols[0]]) 
except ValueError:
    # I want to register on my log the message recived on ORIGINAL VALUE
    mask = pd.to_numeric(pd_csv[principal_cols[0]].fillna('0'), errors='coerce').isna()
    values_in_principal = mask[mask].index.to_list()

In [286]:
value_to_remove = list()
for i in values_in_principal:
    if(pd_csv.loc[i,principal_cols[0]] == '-'):
        values_in_principal.remove(i)

In [287]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,NaN,Common Stock (Fully diluted 43.4%),6350,6350
5,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,20907,20907,NaN,NaN
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,NaN,Member Units (Fully diluted 41.0%),42,3740
9,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,342,4040,NaN,NaN
11,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
12,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,NaN,Warrants (Fully diluted 21.3%),1193,3380
13,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,NaN,Common Stock (Fully diluted 9.8%),1177,1560
14,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,10298,13033,NaN,NaN


In [288]:
## Switch unless there is Maturity Date in the picture
if(len(maturity_cols)<1):
    pd_csv.loc[values_in_principal, [principal_cols[0], type_of_inv_cols[0]]] = pd_csv.loc[values_in_principal, [type_of_inv_cols[0], principal_cols[0]]].values
    last_2_as_nan_list = pd_csv.index[(pd_csv.iloc[:,-1].isnull()) & (pd_csv.iloc[:,-2].isnull())  & (pd_csv.iloc[:,-3].notnull())].to_list()
    #Need to drop last 2 NaN List
    pd_csv.drop(last_2_as_nan_list,inplace=True)
else:
    print("Maturity column present")

In [289]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
3,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
4,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),NaN,6350,6350
7,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
8,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),NaN,42,3740
11,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
12,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),NaN,1193,3380
13,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),NaN,1177,1560
16,Control Investments,"CBT Nuggets, LLC",Produces and Sells IT Training Certification V...,Member Units (Fully diluted 41.6%),NaN,1300,8370
18,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,"14% Secured Debt (Maturity - May 31, 2013)",4000,3997,3997
19,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,Class B Member Units (12% cumulative),NaN,3000,3000


In [290]:
pd_csv[-60:]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
343,Non-ControlNon-Affiliate Investments,Metal Services LLC,Steel Mill Services,"LIBOR Plus 6.50%, Current Coupon 7.75%, Secure...",4988,4894,5054
345,Non-ControlNon-Affiliate Investments,"Metals USA, Inc.",Operator of Metal Service Centers,"LIBOR Plus 5.00%, Current Coupon 6.25%, Secure...",7481,7409,7534
347,Non-ControlNon-Affiliate Investments,Milk Specialties Company,Processor of Nutrition Products,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",5000,4952,5034
349,Non-ControlNon-Affiliate Investments,"Miramax Film NY, LLC",Motion Picture Producer and Distributor,Class B Units (Fully diluted 0.2%),NaN,500,667
351,Non-ControlNon-Affiliate Investments,"Mitel US Holdings, Inc.",Enterprise IP Telephone Provider,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",4000,3960,4013
353,Non-ControlNon-Affiliate Investments,"Mmodal, Inc.",Healthcare Equipment and Services,"LIBOR Plus 5.50%, Current Coupon 6.75%, Secure...",3980,3931,3860
355,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,"LIBOR Plus 9.00%, Current Coupon 10.50%, Secur...",5054,4837,5054
356,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,Warrants (Fully diluted 1.5%),NaN,150,150
359,Non-ControlNon-Affiliate Investments,Mood Media Corporation,Music Programming and Broadcasting,"LIBOR Plus 5.50%, Current Coupon 7.00%, Secure...",1770,1756,1783
361,Non-ControlNon-Affiliate Investments,"National Vision, Inc.",Discount Optical Retailer,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",3209,3165,3249


### Changes for overflowing business description

In [291]:
pd_csv = pd_csv.reset_index(drop=True)

In [292]:
desc_cols = [col for col in pd_csv.columns if 'Description' in col]
desc_cols

['Business Description']

In [293]:
comp_cols = [col for col in pd_csv.columns if 'Company' in col]
comp_cols

['Portfolio Company ']

In [244]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
186,Non-ControlNon-Affiliate Investments,Metal Services LLC,Steel Mill Services,"LIBOR Plus 6.50%, Current Coupon 7.75%, Secure...",4988,4894,5054
187,Non-ControlNon-Affiliate Investments,"Metals USA, Inc.",Operator of Metal Service Centers,"LIBOR Plus 5.00%, Current Coupon 6.25%, Secure...",7481,7409,7534
188,Non-ControlNon-Affiliate Investments,Milk Specialties Company,Processor of Nutrition Products,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",5000,4952,5034
189,Non-ControlNon-Affiliate Investments,"Miramax Film NY, LLC",Motion Picture Producer and Distributor,Class B Units (Fully diluted 0.2%),NaN,500,667
190,Non-ControlNon-Affiliate Investments,"Mitel US Holdings, Inc.",Enterprise IP Telephone Provider,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",4000,3960,4013
191,Non-ControlNon-Affiliate Investments,"Mmodal, Inc.",Healthcare Equipment and Services,"LIBOR Plus 5.50%, Current Coupon 6.75%, Secure...",3980,3931,3860
192,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,"LIBOR Plus 9.00%, Current Coupon 10.50%, Secur...",5054,4837,5054
193,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,Warrants (Fully diluted 1.5%),NaN,150,150
194,Non-ControlNon-Affiliate Investments,Mood Media Corporation,Music Programming and Broadcasting,"LIBOR Plus 5.50%, Current Coupon 7.00%, Secure...",1770,1756,1783
195,Non-ControlNon-Affiliate Investments,"National Vision, Inc.",Discount Optical Retailer,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",3209,3165,3249


In [245]:
### Changes for overflowing business description
business_desc_index = list()
business_desc_temp = ''
i = 0
while(i < len(pd_csv)-1):
    if( pd_csv.loc[i,comp_cols[0]] == pd_csv.loc[i+1,comp_cols[0]] ):
        if((pd_csv.loc[i,desc_cols[0]] != pd_csv.loc[i+1,desc_cols[0]])):
            company_name = pd_csv.loc[i,comp_cols[0]]
            k = 0
            business_desc_temp = ''
            #print(company_name)
            while ( k != -1 and k+i+1<len(pd_csv)):
                #print("K",k)
                if(company_name == pd_csv.loc[k+i,comp_cols[0]]):
                    #print("Company Name",pd_csv.loc[k+i,comp_cols[0]])
                    if(str(pd_csv.loc[k+i,desc_cols[0]]) not in business_desc_temp):
                        business_desc_temp = business_desc_temp+' '+str(pd_csv.loc[k+i,desc_cols[0]])
                    #print(business_desc_temp)
                else:
                    break
                k = k + 1
            index_run = 0
            while(index_run < k):
                pd_csv.loc[index_run+i,desc_cols[0]] = business_desc_temp
                index_run = index_run + 1
            i = i + k
    i = i + 1
        

In [246]:
pd_csv[(pd_csv.iloc[:,-1].isnull()) & (pd_csv.iloc[:,-2].isnull())  & (pd_csv.iloc[:,-3].isnull())]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value


In [247]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
0,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
1,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),NaN,6350,6350
2,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
3,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),NaN,42,3740
4,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
5,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),NaN,1193,3380
6,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),NaN,1177,1560
7,Control Investments,"CBT Nuggets, LLC",Produces and Sells IT Training Certification V...,Member Units (Fully diluted 41.6%),NaN,1300,8370
8,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,"14% Secured Debt (Maturity - May 31, 2013)",4000,3997,3997
9,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,Class B Member Units (12% cumulative),NaN,3000,3000


In [117]:
pd_csv[-60:]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
186,Non-ControlNon-Affiliate Investments,Metal Services LLC,Steel Mill Services,"LIBOR Plus 6.50%, Current Coupon 7.75%, Secure...",4988,4894,5054
187,Non-ControlNon-Affiliate Investments,"Metals USA, Inc.",Operator of Metal Service Centers,"LIBOR Plus 5.00%, Current Coupon 6.25%, Secure...",7481,7409,7534
188,Non-ControlNon-Affiliate Investments,Milk Specialties Company,Processor of Nutrition Products,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",5000,4952,5034
189,Non-ControlNon-Affiliate Investments,"Miramax Film NY, LLC",Motion Picture Producer and Distributor,Class B Units (Fully diluted 0.2%),NaN,500,667
190,Non-ControlNon-Affiliate Investments,"Mitel US Holdings, Inc.",Enterprise IP Telephone Provider,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",4000,3960,4013
191,Non-ControlNon-Affiliate Investments,"Mmodal, Inc.",Healthcare Equipment and Services,"LIBOR Plus 5.50%, Current Coupon 6.75%, Secure...",3980,3931,3860
192,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,"LIBOR Plus 9.00%, Current Coupon 10.50%, Secur...",5054,4837,5054
193,Non-ControlNon-Affiliate Investments,"Modern VideoFilm, Inc.",Post-Production Film Studio,Warrants (Fully diluted 1.5%),NaN,150,150
194,Non-ControlNon-Affiliate Investments,Mood Media Corporation,Music Programming and Broadcasting,"LIBOR Plus 5.50%, Current Coupon 7.00%, Secure...",1770,1756,1783
195,Non-ControlNon-Affiliate Investments,"National Vision, Inc.",Discount Optical Retailer,"LIBOR Plus 5.75%, Current Coupon 7.00%, Secure...",3209,3165,3249


## Changes for END Quarters

In [118]:
if (len(maturity_cols) > 0):

    share_cols = [col for col in pd_csv.columns if 'Share' in col]
    rate_cols = [col for col in pd_csv.columns if 'Rate' in col]

    try:
        for i in range(int(pd_csv.shape[0])):
            for j in range(pd_csv.shape[1]):
                pd_csv.iat[i, j] = re.sub(r"\(\d+\)", "", pd_csv.iat[i, j])
    except:
        print("Bracket digits not removed")

    # Shifting the value when share has % and rate has / and last column is null
    try:
        rate_share_index = pd_csv.index[(pd_csv[share_cols[0]].astype(str).str.contains("%")) & (
            pd_csv[rate_cols[0]].astype(str).str.contains("/")) & (pd_csv.iloc[:, -1].isnull())].to_list()
        pd_csv.loc[rate_share_index, share_cols[0]:] = pd_csv.loc[rate_share_index, share_cols[0]:].shift(1, axis=1)
    except:
        print("No Rate")

    # Shifting the value when share has % and rate has / and maturity column is null
    try:
        rate_share_index = pd_csv.index[(pd_csv[share_cols[0]].astype(str).str.contains("%")) & (
            pd_csv[rate_cols[0]].astype(str).str.contains("/")) & (pd_csv[maturity_cols[0]].isnull())].to_list()
        pd_csv.loc[rate_share_index, share_cols[0]:maturity_cols[0]] = pd_csv.loc[rate_share_index,
                                                                       share_cols[0]:maturity_cols[0]].shift(1,
                                                                                                             axis=1)
    except:
        print("No Rate")

    # Shifting the value when Rate does not have %
    rate_nan_index = pd_csv.index[~pd_csv[rate_cols[0]].astype(str).str.contains("%")].to_list()
    # Shifting the value when maturity is NaN
    maturity_nan_index = pd_csv.index[~pd_csv[maturity_cols[0]].astype(str).str.contains("/")].to_list()

    # Identifying investment date column
    investment_date_cols = [col for col in pd_csv.columns if 'Investment Date' in col]

    # If () in Fair Value, make that np.nan
    try:
        fair_cols = [col for col in pd_csv.columns if 'Fair' in col]
        pd_csv.loc[pd_csv[fair_cols[0]].str.contains('\[(.+)\]|\((.+)\)|\{(.+)\}', na=False), fair_cols[0]] = np.nan
    except:
        print("No Fair Value columns")

    # Selecting the column after rate
    # Then pushing the row based on amount of columns which are Null
    try:
        last_3_nan_list = pd_csv.index[
            (pd_csv.iloc[:, -1].isnull()) & (pd_csv.iloc[:, -2].isnull()) & (pd_csv.iloc[:, -3].isnull())].to_list()
        pd_csv.loc[last_3_nan_list, rate_cols[0]:] = pd_csv.loc[last_3_nan_list, rate_cols[0]:].shift(3, axis=1)
    except:
        print("No Rate")

    # Selecting the column after rate
    # Then pushing the row based on amount of columns which are Null
    try:
        last_2_nan_list = pd_csv.index[(pd_csv.iloc[:, -1].isnull()) & (pd_csv.iloc[:, -2].isnull()) & (
            pd_csv.iloc[:, -3].notnull())].to_list()
        pd_csv.loc[last_2_nan_list, maturity_cols[0]:] = pd_csv.loc[last_2_nan_list, maturity_cols[0]:].shift(2,
                                                                                                              axis=1)
    except:
        print("No Maturity")

    try:
        last_1_nan_list = pd_csv.index[(pd_csv.iloc[:, -1].isnull()) & (pd_csv.iloc[:, -2].notnull()) & (
            pd_csv.iloc[:, -3].notnull())].to_list()
        pd_csv.loc[last_1_nan_list, maturity_cols[0]:] = pd_csv.loc[last_1_nan_list, maturity_cols[0]:].shift(1, axis=1)
    except:
        print("No Rate")

    # Checking for Dates in Rates
    try:
        date_in_rates = pd_csv.index[pd_csv[rate_cols[0]].astype(str).str.contains("/")].to_list()
        # Switching it to Investment Date
        if (pd_csv.columns.get_loc(investment_date_cols[0]) > 4):
            pd_csv.loc[date_in_rates, [rate_cols[0], investment_date_cols[0]]] = pd_csv.loc[
                date_in_rates, [investment_date_cols[0], rate_cols[0]]].values
        else:
            pd_csv.loc[date_in_rates, [rate_cols[0], maturity_cols[0]]] = pd_csv.loc[
                date_in_rates, [maturity_cols[0], rate_cols[0]]].values

    except:
        print("Error in Dates in Rates")
        
    
    # Checking for Dates in Shares
    try:
        date_in_shares = pd_csv.index[pd_csv[share_cols[0]].astype(str).str.contains("/")].to_list()
        # --------------------------------------------------------------------------------------------------------------#
        # Switching it to Investment Date or Maturity Date, should depend on the date, if the date is less than
        if (pd_csv.columns.get_loc(investment_date_cols[0]) > 4):
            pd_csv.loc[date_in_shares, [share_cols[0], investment_date_cols[0]]] = pd_csv.loc[
                date_in_shares, [investment_date_cols[0], share_cols[0]]].values
    except:
        print("Error in Dates in Shares")


    # Checking for Dates in Principal
    date_in_principal = pd_csv.index[pd_csv[principal_cols[0]].astype(str).str.contains("/")].to_list()
    # Switching it to Maturity Date
    pd_csv.loc[date_in_principal, [principal_cols[0], maturity_cols[0]]] = pd_csv.loc[
        date_in_principal, [maturity_cols[0], principal_cols[0]]].values

    # % in Principal Column
    perc_in_principal = pd_csv.index[(pd_csv[principal_cols[0]].astype(str).str.contains("%"))]
    # Switching the rows with principal and rate
    pd_csv.loc[perc_in_principal, [principal_cols[0], rate_cols[0]]] = pd_csv.loc[
        perc_in_principal, [rate_cols[0], principal_cols[0]]].values

    # Switching the rows with principal and shares
    pd_csv.loc[perc_in_principal, [principal_cols[0], share_cols[0]]] = pd_csv.loc[
        perc_in_principal, [share_cols[0], principal_cols[0]]].values

    # Switching from rate to share when rate does not have %
    rate_no_perc = pd_csv.index[
        ~((pd_csv[rate_cols[0]].astype(str).str.contains("%")) | (pd_csv[rate_cols[0]].isnull()))].to_list()

    pd_csv.loc[rate_no_perc, [share_cols[0], rate_cols[0]]] = pd_csv.loc[
        rate_no_perc, [rate_cols[0], share_cols[0]]].values
    # Seeing when to swtich from share to rate

    # If it has % in Share
    share_has_perc = pd_csv.index[(pd_csv[share_cols[0]].astype(str).str.contains("%"))]
    share_to_rate = list()
    for i in share_has_perc:
        # print(pd_csv.loc[i,fair_cols[0]])
        share_string = pd_csv.loc[i, share_cols[0]]
        # If there is PIK in Share
        if ('PIK' in share_string):
            share_to_rate.append(i)
        else:
            # If the digit has more than two digits after decimal in Share
            share_string = share_string.replace("%", "")
            if (str(share_string)[::-1].find('.') == 2):
                share_to_rate.append(i)

    pd_csv.loc[share_to_rate, [share_cols[0], rate_cols[0]]] = pd_csv.loc[
        share_to_rate, [rate_cols[0], share_cols[0]]].values

In [119]:
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]


In [120]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
0,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
1,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),NaN,6350,6350
2,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
3,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),NaN,42,3740
4,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
5,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),NaN,1193,3380
6,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),NaN,1177,1560
7,Control Investments,"CBT Nuggets, LLC",Produces and Sells IT Training Certification V...,Member Units (Fully diluted 41.6%),NaN,1300,8370
8,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,"14% Secured Debt (Maturity - May 31, 2013)",4000,3997,3997
9,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,Class B Member Units (12% cumulative),NaN,3000,3000


In [121]:
last_nan = list()
for i in range(int(pd_csv.shape[0])):
    if(pd_csv.iat[i,pd_csv.shape[1]-1] == ' '):
        last_nan.append(i)
        pd_csv.iat[i,pd_csv.shape[1]-1] = np.nan

In [122]:
pd_csv.loc[last_nan]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value


In [123]:
pd_csv.isnull().sum()/len(pd_csv)

0
Levels                  0.000000
Portfolio Company       0.000000
Business Description    0.000000
Type of Investment      0.000000
Principal               0.345528
Cost                    0.000000
Fair Value              0.000000
dtype: float64

In [124]:
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
pd_csv = pd_csv.dropna(axis=1,thresh=min_count)

In [125]:
pd_csv.isnull().sum()/len(pd_csv)

0
Levels                  0.000000
Portfolio Company       0.000000
Business Description    0.000000
Type of Investment      0.000000
Principal               0.345528
Cost                    0.000000
Fair Value              0.000000
dtype: float64

In [126]:
pd_csv

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
0,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
1,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),NaN,6350,6350
2,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
3,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),NaN,42,3740
4,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
...,...,...,...,...,...,...,...
241,Non-ControlNon-Affiliate Investments,Wilton Brands LLC,Specialty Housewares Retailer,"LIBOR Plus 6.25%, Current Coupon 7.50%, Secure...",1950,1914,1978
242,Non-ControlNon-Affiliate Investments,Wireco Worldgroup Inc.,Manufacturer of Synthetic Lifting Products,"LIBOR Plus 4.75%, Current Coupon 6.00%, Secure...",2488,2466,2522
243,Non-ControlNon-Affiliate Investments,Zilliant Incorporated,Price Optimization and Margin Management Solut...,"12% Secured Debt (Maturity - June 15, 2017)",8000,6911,6911
244,Non-ControlNon-Affiliate Investments,Zilliant Incorporated,Price Optimization and Margin Management Solut...,Warrants (Fully diluted 3.0%),NaN,1071,1071


In [127]:
pd_csv = pd_csv.fillna("")
try:
    for i in range(int(pd_csv.shape[0])):
        for j in range(pd_csv.shape[1]):
            pd_csv.iat[i,j] = re.sub(r"\(\d+\)", "", pd_csv.iat[i,j])
except:
    print("Bracket digits not removed")

In [128]:
second_header = list()
#Identifying where the second header is
for index, row in pd_csv.iterrows():
    if index==1:
        continue
    if 'Portfolio Company' in row[0]:
        second_header.append(index)
#Dropping the second header
try:
    print("Second Header",second_header)
    pd_csv.drop(second_header, inplace=True)
except:
    print("No second header")

Second Header []


## 

In [129]:
pd_csv[:60]

,Levels,Portfolio Company,Business Description,Type of Investment,Principal,Cost,Fair Value
0,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,"12% Secured Debt (Maturity - December 28, 2017)",14750,14557,14557
1,Control Investments,"Bond-Coat, Inc.",Casing and Tubing Coating Services,Common Stock (Fully diluted 43.4%),,6350,6350
2,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,"12% Secured Debt (Maturity - April 20, 2013)",300,300,300
3,Control Investments,"Café Brazil, LLC",Casual Restaurant Group,Member Units (Fully diluted 41.0%),,42,3740
4,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,"12% Secured Debt (Maturity - October 17, 2015)",8103,7928,8093
5,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Warrants (Fully diluted 21.3%),,1193,3380
6,Control Investments,"California Healthcare Medical Billing, Inc.",Outsourced Billing and Revenue Cycle Management,Common Stock (Fully diluted 9.8%),,1177,1560
7,Control Investments,"CBT Nuggets, LLC",Produces and Sells IT Training Certification V...,Member Units (Fully diluted 41.6%),,1300,8370
8,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,"14% Secured Debt (Maturity - May 31, 2013)",4000,3997,3997
9,Control Investments,"Ceres Management, LLC (Lambs Tire & Automotive)",Aftermarket Automotive Services Chain,Class B Member Units (12% cumulative),,3000,3000


In [130]:
len(pd_csv)

246

In [131]:
pd_csv.columns

Index(['Levels', 'Portfolio Company ', 'Business Description',
       'Type of Investment  ', 'Principal ', 'Cost ', 'Fair Value'],
      dtype='object', name=0)

In [132]:
pd_csv.to_csv("Testing.csv")